# Import Data

In [3]:
import scipy.io as sio
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from ModulationPredictionCNN import ModulationPredictionCNN
model_type = 'relu'
model = ModulationPredictionCNN(activation=model_type)

#from ModulationPredictionResNet import ModulationPredictionResNet
#model = ModulationPredictionResNet()
#model_type = 'resnet'

IQSamples = sio.loadmat('ModulationData.mat')
LBSamples = sio.loadmat('Labels.mat')
data = IQSamples['frameswhole']
label =  LBSamples['IQDataLabel']
n_samples = data.shape[-1]
data = np.squeeze(np.moveaxis(data, [0, 1, 2, 3], [-3, -2, -1, -4]))

label_dict = {'16QAM ':0, '64QAM ':1, '8PSK  ':2, 'B-FM  ':3, 'BPSK  ':4, 'CPFSK ':5, 'DSB-AM':6, 'GFSK  ':7,
 'PAM4  ':8, 'QPSK  ':9, 'SSB-AM':10}
lbl = np.zeros(n_samples)
for i in range(n_samples):
    lbl[i] = label_dict[label[i]]
label = lbl

In [4]:
def label2onehot(lbl):
    if len(lbl.shape) == 1:
        d = np.zeros((lbl.size, lbl.max() + 1))
        d[np.arange(lbl.size), lbl] = 1
    else:
        d = lbl
    return d

def onehot2label(d):
    if len(d.shape) == 1:
        lbl = d
    else:
        lbl = d.argmax(axis=1)
    return lbl

label = onehot2label(label)
# Split test data
x_data, x_test, y_data, y_test = train_test_split(data, label, test_size=0.2, random_state=1)
# Split validation data
x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.2, random_state=1)


train_set = {'data':torch.tensor(x_train).float(), 'labels':torch.tensor(y_train).float() }
val_set = {'data':torch.tensor(x_val).float(), 'labels':torch.tensor(y_val).float()}

# Model Accuracy

In [7]:
from ModulationPredictionCNN import ModulationPredictionCNN
import torch
import torch.nn as nn
import os
import pickle

def calculate_accuracy(model, data, label, batch_size, computing_device):
    n_samples = data.shape[0]
    n_minibatch = int((n_samples+batch_size-1)/batch_size)
    accuracy = 0
    I = np.arange(n_samples)
    for i in range(n_minibatch):
        idx = I[batch_size*i:min(batch_size*(i+1), n_samples)]
        dt = data[idx].to(computing_device)
        lbl = label[idx]
        output = model(dt).detach()
        output = output.cpu().numpy()
        output = np.argmax(output,axis=1)

        accuracy += np.sum(output == lbl)

    return accuracy/n_samples

model_file = 'Models/'+model_type+'_model.pt'

batch_size = 1000
computing_device = torch.device("cuda")
#model = ModulationPredictionResNet()
model.load_state_dict(torch.load(model_file))
model.to(computing_device)
    
x_test = torch.tensor(x_test).float().to(computing_device)
accuracy = calculate_accuracy(model, x_test, y_test, batch_size, computing_device)
print('Test Accuracy = ', accuracy)

x_train = torch.tensor(x_train).float().to(computing_device)
accuracy = calculate_accuracy(model, x_train, y_train, batch_size, computing_device)
print('Train Accuracy = ', accuracy)


Test Accuracy =  0.6938636363636363
Train Accuracy =  0.6962784090909091


In [ ]:
loss_file = model_file.strip('.pt') + '_loss.pkl'
accuracy_file = model_file.strip('.pt') + '_accuracy.pkl'
if os.path.isfile(model_file):
    # Load Loss and Accuracy
    with open(loss_file, 'rb') as handle:
        Loss = pickle.load(handle)
    with open(accuracy_file, 'rb') as handle:
        Accuracy = pickle.load(handle)
    n_prev_epochs = len(Loss['train'])
else:
    raise('Error: no loss or accuracy data')

# Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
%matplotlib inline
import matplotlib.pyplot as plt


def get_output(model, data, batch_size, computing_device):
    n_samples = data.shape[0]
    n_minibatch = int((n_samples+batch_size-1)/batch_size)
    accuracy = 0
    I = np.arange(n_samples)
    Output = []
    for i in range(n_minibatch):
        idx = I[batch_size*i:min(batch_size*(i+1), n_samples)]
        dt = data[idx].to(computing_device)
        out = model(dt).detach()
        out = out.cpu().numpy()
        out = np.argmax(out,axis=1)
        Output.extend(list(out))

    return np.asarray(Output)

y_pred = get_output(model, torch.tensor(x_test).float(), 1000, computing_device)
print(y_pred.shape)
confusion_mat = confusion_matrix(y_test, y_pred)
confusion_mat = confusion_mat.astype(float)
print(confusion_mat.dtype)
for r in range(11):
    n_r = y_test[y_test == r].shape[0]
    for c in range(11):
        confusion_mat[r,c] = confusion_mat[r,c]/n_r


plt.imshow(confusion_mat, cmap='coolwarm', interpolation='nearest')
plt.title('Confusion Matrix')
plt.xticks(np.arange(11), label_dict.keys(), rotation=300)
plt.yticks(np.arange(11), label_dict.keys(), rotation=0)
plt.xlabel('Predicted Class')
plt.ylabel('True Class')
plt.colorbar()
plt.savefig(model_type+'_model_conf',  bbox_inches = 'tight')
plt.show()